# Calculate (power-law) creep envelopes

Copyright 2020 Marco A. Lopez-Sanchez

Mozilla Public License 2.0: https://www.mozilla.org/en-US/MPL/2.0/

> **Goal of the notebook**: Understand flow laws and estimate power-law creep envelopes for different rock types.